In [12]:
import urllib.parse

path = './s288c_n20.gff'

with open(path) as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('#'): 
            continue
        s = line.split('\t')
        items = s[8].split(';')
        tags = dict([tmp.split('=') for tmp in items])
        
        if 'Note' in tags:
            print(urllib.parse.unquote(tags['Note']))

TEL01L; Telomeric region on the left arm of Chromosome I; composed of an X element core sequence, X element combinatorial repeats, and a short terminal stretch of telomeric repeats
ARS102~Autonomously Replicating Sequence
hypothetical protein; member of the seripauperin multigene family encoded mainly in subtelomeric regions
hypothetical protein; identified by gene-trapping, microarray-based expression analysis, and genome-wide homology searching


In [13]:
with open(path) as f:
    f.readline()
    f.readline()
    for line in f:
        line = line.rstrip()
        print(line)

#!processor NCBI annotwriter
#!genome-build R64
#!genome-build-accession NCBI_Assembly:GCF_000146045.2
#!annotation-source SGD R64-2-1
##sequence-region NC_001133.9 1 230218
##species https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=559292
NC_001133.9	RefSeq	region	1	230218	.	+	.	ID=id0;Dbxref=taxon:559292;Name=I;chromosome=I;gbkey=Src;genome=chromosome;mol_type=genomic DNA;strain=S288C
NC_001133.9	RefSeq	telomere	1	801	.	-	.	ID=id1;Dbxref=SGD:S000028862;Note=TEL01L%3B Telomeric region on the left arm of Chromosome I%3B composed of an X element core sequence%2C X element combinatorial repeats%2C and a short terminal stretch of telomeric repeats;gbkey=telomere
NC_001133.9	RefSeq	origin_of_replication	707	776	.	+	.	ID=id2;Dbxref=SGD:S000121252;Note=ARS102~Autonomously Replicating Sequence;gbkey=rep_origin
NC_001133.9	RefSeq	gene	1807	2169	.	-	.	ID=gene0;Dbxref=GeneID:851229;Name=PAU8;end_range=2169,.;gbkey=Gene;gene=PAU8;gene_biotype=protein_coding;locus_tag=YAL068C;partial=tr

In [16]:
output = './output/test1.txt'

out = 'Hello World!\n'
with open(output, 'x') as f:
    f.write(out)

FileNotFoundError: [Errno 2] No such file or directory: './output/test1.txt'

In [19]:
import re

path = './SRR453566.sam'

TATA = re.compile('TATA[AT]AA[GA]')
TATA_rev = re.compile('[CT]TT[AT]TATA')

with open(path) as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('@'):
            continue
        s = line.split('\t')
        sequence = s[9]

        m = TATA.search(sequence)

        if m:
            print(m.start(), m.end(), m.string[m.start():m.end()])
        
        m = TATA_rev.search(sequence)

        if m:
            print(m.start(), m.end(), m.string[m.start():m.end()])

54 62 TATATAAG
14 22 TTTTTATA
54 62 TATATAAA
89 97 TTTTTATA
62 70 TTTTTATA
9 17 TTTTTATA
45 53 TATAAAAA
69 77 TATATAAA
75 83 TTTTTATA


In [22]:
class Fasta:
    def __init__(self, id_, description, seq):
        self.id = id_
        self.description = description
        self.seq = seq

    def get_gc_content(self):
        g_count = self.seq.count('G')
        c_count = self.seq.count('C')
        gc_count = (g_count + c_count) / len(self.seq)
        return gc_count

In [23]:
fasta = Fasta('gene01', 'nucleotide sequence of tRNA-Ser',
              'TGGAGTGTTGTCCGAGCGGCTGAAGGAGCATGATTGGAAATCATGTATACGGGTAAATACCTGTATCGAGGGTTCAAATCCCTCACACTCCGT')

In [26]:
fasta.get_gc_content()

0.4838709677419355

In [27]:
def read_fasta(file_name):
     with open(file_name) as fh:
         line = next(fh)
         seq_id, description = line.strip('>\n').split(' ',1)
         seq = ''
         for line in fh:
             if line.startswith('>'):
                 yield Fasta(seq_id, description, seq)
                 seq_id, description = line.strip().strip('>').split(' ', 1)
                 seq = ''
             else:
                 seq += line.strip('\n').upper()
         yield Fasta(seq_id, description, seq)